In [198]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import config
import mysql.connector

In [11]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password)

In [6]:
cursor = cnx.cursor()

## Create "music" database to dump Spotify information into

In [36]:
stmt = """CREATE DATABASE music DEFAULT CHARACTER SET 'utf8'"""
cursor.execute(stmt)

InternalError: Unread result found

## Connect to "music" database

In [214]:
cnx = mysql.connector.connect(
    host = config.host,
    user = config.user,
    passwd = config.password,
    database = 'music')

In [215]:
cursor = cnx.cursor()

In [63]:
tables = {}
tables['artists'] = (
    "CREATE TABLE artists ("
    "  artist_id varchar(50) NOT NULL,"
    "  name varchar(100) NOT NULL,"
    "  genre varchar(100) NOT NULL,"
    "  popularity int(3),"
    "  PRIMARY KEY (artist_id)"
    ") ENGINE=InnoDB")

In [64]:
tables['albums'] = (
    "CREATE TABLE albums ("
    "  album_id varchar(50) NOT NULL,"
    "  album_name varchar(100) NOT NULL,"
    "  artist_id1 varchar(50) NOT NULL,"
    "  artist_id2 varchar(50),"
    "  artist_id3 varchar(50),"
    "  release_date DATE,"
    "  total_tracks int(2),"
    "  PRIMARY KEY (album_id)"
    ") ENGINE=InnoDB")

In [65]:
tables['songs'] = (
    "CREATE TABLE songs ("
    "  song_id varchar(50) NOT NULL,"
    "  song_name varchar(100) NOT NULL,"
    "  artist_id1 varchar(50) NOT NULL,"
    "  artist_id2 varchar(50),"
    "  artist_id3 varchar(50),"
    "  album_id varchar(50),"
    "  track_number int(2),"
    "  popularity int(2),"
    "  PRIMARY KEY (song_id)"
    ") ENGINE=InnoDB")

In [66]:
tables['features'] = (
    "CREATE TABLE features ("
    "  song_id varchar(50) NOT NULL,"
    "  feature_id varchar(100) NOT NULL,"
    "  danceability float,"
    "  energy float,"
    "  key_ float,"
    "  loudness float,"
    "  mode float,"
    "  speechiness float,"
    "  acousticness float,"
    "  instrumentalness float,"
    "  liveness float,"
    "  valence float,"
    "  tempo float,"
    "  duration_ms float,"
    "  time_signature float,"
    "  PRIMARY KEY (feature_id)"
    ") ENGINE=InnoDB")

In [67]:
tables['sections'] = (
    "CREATE TABLE sections ("
    "  song_id varchar(50) NOT NULL,"
    "  section_id varchar(100) NOT NULL,"
    "  start float(10),"
    "  duration float(10),"
    "  confidence float(10),"
    "  loudness float(10),"
    "  tempo float(10),"
    "  tempo_confidence float(10),"
    "  key_ float(10),"
    "  key_confidence float(10),"
    "  mode float(10),"
    "  mode_confidence float(10),"
    "  time_signature float(10),"
    "  time_signature_confidence float(10),"
    "  PRIMARY KEY (section_id)"
    ") ENGINE=InnoDB")

In [68]:
for table_name in tables:
    table_description = tables[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

Creating table artists: OK
Creating table albums: OK
Creating table songs: OK
Creating table features: OK
Creating table sections: OK


## Connect to Spotfiy API to add information into database

In [201]:
client_credentials_manager = SpotifyClientCredentials(client_id='40f2cc0372324a4c85b28e189aa51f81', client_secret='d8dde2eeaa044974bfbda56e9cb76c7f')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
## get artist URI from Spotify desktop app to query API
jayz_uri = 'spotify:artist:3nFkdlSjzX9mRTtwJOzDYB'
kanye_uri = 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'
rolling_stones_uri = 'spotify:artist:22bE4uQ6baNwSHPVcDxLCe'
bruce_springsteen_uri = 'spotify:artist:3eqjTLE0HfPfh78zjh6TqT'
red_hot_chilli_peppers_uri = 'spotify:artist:0L8ExT028jH3ddEcZwqJJ5'
kendrick_uri = 'spotify:artist:2YZyLoL8N0Wb9xBt1NhZWg'
fleetwood_mac_uri = 'spotify:artist:08GQAI4eElDnROBrJRGE0X'

uris = [jayz_uri, kanye_uri, rolling_stones_uri, bruce_springsteen_uri, red_hot_chilli_peppers_uri, kendrick_uri, fleetwood_mac_uri]


In [10]:
# Get artist information from API

def get_artist(uri):
   
    artist = sp.artist(uri)
    artist_dict = {'artist_id' : artist['id'],
          'name' : artist['name'],
          'genre' : (', ').join(artist['genres']),
          'popularity' : artist['popularity']      
    }
    return artist_dict

    


In [11]:
# Prepare artist information for insert into SQL database

artists= []
for artist in uris:
    artists.append(get_artist(artist))
    
insert_tuples = [tuple(x.values()) for x in artists]

In [13]:
# Build insert statement and insert artist information into table

cols_insert = ''
str_insert = ''
for col in artists[0].keys():
    cols_insert += col + ', '
    str_insert += '%s, '
cols_insert = cols_insert.strip(', ')
str_insert = str_insert.strip(', ')

insert_statemet = """INSERT INTO music.artists ({}) VALUES ({})""".format(cols_insert, str_insert)

cursor.executemany(insert_statemet, insert_tuples)
cnx.commit()

## Use artist ids to query each artist's albums to insert into the database

In [64]:
stmt = """SELECT artist_id FROM music.artists"""
cursor.execute(stmt)

artist_ids = [x[0] for x in cursor.fetchall()]


In [82]:
artist_uris = ['spotify:artist:'+ artist_id for artist_id in artist_ids]
 


In [58]:
# hova_albums = sp.artist_albums(artist_id=jayz_uri, limit=45, country='US')['items']

def parse_album(album):

        album_dict = {'album_id' : album['id'],
                     'album_name' : album['name'],
                     }
        if len(album['artists']) > 3:
            album_dict['artist_id1'] = album['artists'][0]['id']
            album_dict['artist_id2'] = album['artists'][1]['id']
            album_dict['artist_id3'] = album['artists'][2]['id']
        elif len(album['artists']) > 1:
            album_dict['artist_id1'] = album['artists'][0]['id']
            album_dict['artist_id2'] = album['artists'][1]['id']
            album_dict['artist_id3'] = None
        else:
            album_dict['artist_id1'] = album['artists'][0]['id']
            album_dict['artist_id2'] = None
            album_dict['artist_id3'] = None
            
        album_dict['release_date']= album['release_date']
        album_dict['total_tracks'] = album['total_tracks']
        
        return album_dict


In [97]:
all_albums = []
for artist in artist_uris:
    albums = sp.artist_albums(artist_id=artist, limit=50, country='US')['items']
    all_albums.append(parse_album(albums[0]))
    for album in albums[1:]:
        if album['album_type'] != 'album':
            continue
        elif album['name'] not in [x['album_name'] for x in all_albums]:
            all_albums.append(parse_album(album))
        else:
            continue
    

            

In [103]:
for album in all_albums:
    if album['artist_id1'] == '0LyfQWJT6nXafLPZqxe9Of':
        all_albums.remove(album)
    else:
        continue
all_albums

[{'album_id': '3mlM7EnbGFBqcYNZ32ULWh',
  'album_name': 'Before the Beginning - 1968-1970 Rare Live & Demo Sessions (Remastered)',
  'artist_id1': '08GQAI4eElDnROBrJRGE0X',
  'artist_id2': None,
  'artist_id3': None,
  'release_date': '2019-11-15',
  'total_tracks': 41},
 {'album_id': '06JqOkwwy91OxrApXclzYf',
  'album_name': "50 Years - Don't Stop",
  'artist_id1': '08GQAI4eElDnROBrJRGE0X',
  'artist_id2': None,
  'artist_id3': None,
  'release_date': '2018-11-16',
  'total_tracks': 50},
 {'album_id': '6dMdadS6Ng71pD9As93ufx',
  'album_name': 'Say You Will',
  'artist_id1': '08GQAI4eElDnROBrJRGE0X',
  'artist_id2': None,
  'artist_id3': None,
  'release_date': '2003-04-15',
  'total_tracks': 18},
 {'album_id': '6YEnWIpSS19x0lKzJPi9Ej',
  'album_name': "Shrine '69",
  'artist_id1': '08GQAI4eElDnROBrJRGE0X',
  'artist_id2': None,
  'artist_id3': None,
  'release_date': '2000-04-28',
  'total_tracks': 10},
 {'album_id': '70MrKoi2AdCFBxKqeXDPHZ',
  'album_name': 'The Dance',
  'artist_id1

In [106]:
# Prepare artist information for insert into SQL database
    
insert_tuples = [tuple(x.values()) for x in all_albums]


# Build insert statement and insert artist information into table

cols_insert = ''
str_insert = ''
for col in all_albums[0].keys():
    cols_insert += col + ', '
    str_insert += '%s, '
cols_insert = cols_insert.strip(', ')
str_insert = str_insert.strip(', ')

insert_statemet = """INSERT INTO music.albums ({}) VALUES ({})""".format(cols_insert, str_insert)

cursor.executemany(insert_statemet, insert_tuples)
cnx.commit()

## Pull tracks from albums in database to insert track information for each song

In [109]:
stmt = """SELECT album_id FROM music.albums"""
cursor.execute(stmt)

album_ids = [x[0] for x in cursor.fetchall()]


In [121]:
## Parse songs from album tracks for the relevant information to be uploaded into the DB

def parse_songs(album_id):
    album_tracks = sp.album_tracks(album_id)
    songs = []
    for song in album_tracks['items']:
        song_dict = {'song_id' : song['id'],
                     'song_name' : song['name'],
                     }
        if len(song['artists']) > 3:
            song_dict['artist_id1'] = song['artists'][0]['id']
            song_dict['artist_id2'] = song['artists'][1]['id']
            song_dict['artist_id3'] = song['artists'][2]['id']
        elif len(song['artists']) > 1:
            song_dict['artist_id1'] = song['artists'][0]['id']
            song_dict['artist_id2'] = song['artists'][1]['id']
            song_dict['artist_id3'] = None
        else:
            song_dict['artist_id1'] = song['artists'][0]['id']
            song_dict['artist_id2'] = None
            song_dict['artist_id3'] = None
            
        song_dict['album_id']= album_id
        song_dict['track_number'] = song['track_number']
        
        if 'popularity' in song.keys():
            song_dict['popularity'] = song['popularity']
        else:
            song_dict['popularity'] = None
        
        songs.append(song_dict)
            
    return songs


In [133]:
all_songs = []

for album in album_ids:
    all_songs.append(parse_songs(album))

flat_songs = [item for album in all_songs for item in album]

In [135]:
# Prepare artist information for insert into SQL database
    
insert_tuples = [tuple(x.values()) for x in flat_songs]


# Build insert statement and insert artist information into table

cols_insert = ''
str_insert = ''
for col in flat_songs[0].keys():
    cols_insert += col + ', '
    str_insert += '%s, '
cols_insert = cols_insert.strip(', ')
str_insert = str_insert.strip(', ')

insert_statemet = """INSERT INTO music.songs ({}) VALUES ({})""".format(cols_insert, str_insert)

cursor.executemany(insert_statemet, insert_tuples)
cnx.commit()

## Use song ids to pull features for each song to upload into db

In [202]:
stmt = """SELECT song_id FROM music.songs"""
cursor.execute(stmt)

song_ids = [x[0] for x in cursor.fetchall()]


In [164]:


def parse_features(song):
    song_dict = sp.audio_features(song)[0]
    features = {'song_id' : song,
                'feature_id' : song_dict['id'] + str(song_dict['duration_ms']),
                'danceability' : song_dict['danceability'],
                'energy' : song_dict['energy'],
                'key_' : song_dict['key'],
                'loudness' : song_dict['loudness'],
                'mode' : song_dict['mode'],
                'speechiness' : song_dict['speechiness'],
                'acousticness' : song_dict['acousticness'],
                'instrumentalness' : song_dict['instrumentalness'],
                'liveness' : song_dict['liveness'],
                'valence' : song_dict['valence'],
                'tempo' : song_dict['tempo'],
                'duration_ms' : song_dict['duration_ms'],
                'time_signature' : song_dict['time_signature']
        
    }
    return features
    

In [166]:
song_features = []
for song in song_ids:
    song_features.append(parse_features(song))


retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs
retrying ...1secs


In [169]:
# Prepare artist information for insert into SQL database
    
insert_tuples = [tuple(x.values()) for x in song_features]


# Build insert statement and insert artist information into table

cols_insert = ''
str_insert = ''
for col in song_features[0].keys():
    cols_insert += col + ', '
    str_insert += '%s, '
cols_insert = cols_insert.strip(', ')
str_insert = str_insert.strip(', ')

insert_statemet = """INSERT INTO music.features ({}) VALUES ({})""".format(cols_insert, str_insert)

cursor.executemany(insert_statemet, insert_tuples)
cnx.commit()

## Use song ids to query sections from the spotify api to upload section information into the database

In [203]:
def get_sections(song):
    sections = sp.audio_analysis(song)['sections']
    sections_list = []
    for section in sections:
                        
                        sections_list.append(
                        {'song_id' : song,
                        'section_id' : song + str(section['start']),
                        'start' : section['start'],
                        'duration': section['duration'],
                        'confidence': section['confidence'],
                        'loudness' : section['loudness'],
                        'tempo' : section['tempo'],
                        'tempo_confidence' : section['tempo_confidence'],
                        'key_' : section['key'],
                        'key_confidence' : section['key_confidence'],
                        'mode' : section['mode'],
                        'mode_confidence' : section['mode_confidence'],
                        'time_signature' : section['time_signature'],
                        'time_signature_confidence' : section['time_signature_confidence']})
        
    return sections_list

In [209]:
all_sections = []
count = 0
for song in song_ids:
    all_sections.append(get_sections(song))
    count += 1
    print(count, song)

1 00Fboe8pNoWHLg1nn3e7Oi
2 00hKh3kUrEgNSwPdfCmu2X
3 01Ad3YljWv9gfMVYpn6cL7
4 01fNwidwuko3gFvAhHKrZ6
5 01hkFX17883ndEMW3N8gVT
6 01JYvQ6udfhUw9Jed9EzoK
7 01kYL8egWVVF09irxev3dh
8 01tBpn6TR9LTG1xbwnXUOC
9 01VtifRjNkyUZbxdPwwQTb
10 021JldNpXGATNSC69edcBT
11 02hvHNT0oUdqNnf5spcgPG
12 02jrZxFA2nMEweoUc5UvX3
13 02LAK7qT1wya0klSeNO96f
14 02lWbJ8NQ7IZGuNGfG34tE
15 02uosCE5o2nrozzl6EazOj
16 02v6roMQ0vd5316rukoUeh
17 02Y7UtZLh7SbhYsYG5N7Rr
18 03AbxguWXpMnkR7HnSlxKG
19 03dILakIqUVrgHwtrXlvHg
20 03h0xXtdHey0y1oijFxyz7
21 03QlwMUdrHSxj5P5vZ0A4T
22 03UGZcl5rbmGgZktXbHnmc
23 03zVignBAdwH14qxChAY0x
24 04gJ8tbd8psGJDZVxneeJp
25 05BcisbhFsmVpTZ0kGD7gK
26 05eKAYwnkIOJIwcZSJSzRs
27 05Iawel3maRW3mSHYmXxMZ
28 05icsRv9AUQHO7GaJszvDx
29 05oETzWbd4SI33qK2gbJfR
30 05rizAAJYYh9y2SnAIoCPZ
31 05Va2Xt5QZInHtqAzlm7R8
32 05XKLYOAsDeZyA5983IHno
33 068K0Pa4o6aWZzefuxJ8v3
34 06c08ve2aODGVHZwjWTxsR
35 06FCvd7rrRcF3DdvWH5Isp
36 06h9kk12VjJ2bqcBc6IScR
37 06OV3qRCboo6THc73Vy2l5
38 06Rytq5WaRf15wr80moAhX
39 06wTEKL2rSrSaOjFtg

309 0Wqhgufw1YY63QivQfNUtD
310 0wQzwIo50V1Uboha0bymhg
311 0wraBWkoBLp46K8AdQO9ah
312 0wVliUg2jY2zvXbM7C7Tix
313 0X2E9aiHi70UlTOW7OiXfL
314 0xAjq4KAQdJvUyCM7fFZ9K
315 0XCEjoymU5mjVU4J2TlDnP
316 0xCgcDZnAiCD35gZkx1wsP
317 0XerdRuWBYMi4PmYmcN4j5
318 0XrPZPotxZNDkuvauuVO8R
319 0xSvfXYyDqQ3sncpGveHbg
320 0Xt6VICfhanGh5l0YJUIvf
321 0xULmONvBMOspIStFLs5j4
322 0Xx69L0ebRvnnv0NruzJYa
323 0y2DBEp460zmIdothHGaXJ
324 0Y8kROhHXpVgGzwzxfWiBs
325 0y9uTzK9cNKSAEHnpeRG8C
326 0yaYq5I0HyUcuR4U073Vf2
327 0YdL0lrO1K6XAR6Q1SJ6SD
328 0Ye9OGxiHF3jylblTCuX7Q
329 0yhxBvedRdGxsPZHJNI4VA
330 0YM8m2dJ9DKiUO6ZnGzYu6
331 0YosGlYPvTdZmUzMKiFvta
332 0YsBmhlgivhVxWytwq0As4
333 0YTKU34LVbPMKqjk0w9TXU
334 0ytvC9tG03obop9RXqG3af
335 0YuCATlVwxG5Mq8lN2Omro
336 0yvpBvhz7Xft2zJi8DfKEo
337 0Z0fC0YNVyikjF50WMRdgS
338 0zaqvXwFoLKCPiWc0neuAr
339 0zAr5DOACdbJQHRYfPRvWn
340 0ZeQYobHf2kr2fkZOjhfWH
341 0zgfsOvlwyTDT5HEGXMXBs
342 0ZHu7jkSSrT0eK4OxuG4O5
343 0zLClc0emc6qUeV1p5nc99
344 0zlYK8bBnZYkINDnCcPc4z
345 0zMgwXDyZ5qaqXMLW3h4lr
3

613 1sTeIjsjonMTTQyffRgehj
614 1suHwrkobDEtey9rQ5wXEh
615 1SVTZJAXslouRaIsxmdUz7
616 1T23vUhXAwRidpuk6IsnQP
617 1t27qadlhIxuTPgTN05nAt
618 1T9OPU2WZfaMTthbBILrz7
619 1TczOoTLmJ65u5kIJrRqM8
620 1TgysUGMi7EaFI1yqhcjUA
621 1th3G3okofWlvGWAAR7Y4V
622 1u0kS2ZMHHSuPiadDJ0eyk
623 1U13SSgDOxK98L1FTWYrjQ
624 1UGD3lW3tDmgZfAVDh6w7r
625 1Ulh5rWbhhV3PltDadTBZC
626 1UMV04lFJagdJVWetpb5A8
627 1UtHyDuh9FijqGHcLgAKUm
628 1UUpCSq4WNvhQdGZNpo7sG
629 1UXII0VskNcRP9rJKfeggC
630 1v1ZbS1ARt1rez9lrXbze2
631 1Vb1iR09yHdwLO7Z7YSKcg
632 1VEGh373Jf1jaL5vlMrCIu
633 1VkhHPBER6TR0yfk7aZrgu
634 1VN2vWSkSmMKOhxr8lHzSx
635 1Vp4St7JcXaUoJcIahtf3L
636 1Vp8U39YNsDfd6yVuaUq12
637 1vPOt7hpjQ9sg1Y1u5fmIY
638 1vqZh6D1JlnsIikyvDThfc
639 1VVVFRDykk4pLQMGssCJWp
640 1VWOGyCnOzOO7NhfCRUIPn
641 1VyDQtfhX6gqhWTGU7Oe8X
642 1w29AQqTc0y0VfA0ETJtt3
643 1W5BzwkPzhgLlYRqUALLw5
644 1W8HkY1S5nRQqaDdGiCNwr
645 1wBEOTocwt9ExWNHhT3mub
646 1wbrc6umQQ3StMmSF1uKEY
647 1WF2mTndMlNq2goeUetYLk
648 1WLBuLK0LQSdG1MPLhljjg
649 1WlCBgVSr5dKErEvNjX5B7
6

917 2PieESXphabz5AkALFS08X
918 2Pkq3etYG5Yfm7Gof55oNf
919 2PlPegkfW8hx7HLfOlWrz2
920 2pTkbTFJ6QAVAuVzdfUzU4
921 2Q5g6lSz6tRBkkSmxud3FK
922 2qBDiLqXtqffU7pIvIams0
923 2QENt8VeHEO6ICUxs24OhE
924 2QOMGq8wVTZbLmh7McrvgF
925 2QpGZOhTCHHiKmpSO9FW4h
926 2QVYTN3Z0IGLVF7YEQN5EJ
927 2qZICZ1FBFd8HBv88UJ2J8
928 2QzPc7eFqnxNSGneit1zC7
929 2r41JRUZ5Cjcx98ARVI0vo
930 2R6go62CuxqqX0w1TgXxes
931 2Ra0demNPBUydUaJWTMsy6
932 2rDdGYpZ69iqfrGGrEwq3P
933 2rFoyoRGNu5XiHngfVLpKL
934 2rGld3cZITsCvcjFf8AZHJ
935 2rhS1u6ckiW3fSslPn5V3A
936 2rJIvMcFLEBV2g1UimJcdL
937 2rLnM6RGAdRAcVeIOXhnAg
938 2RRFTLIglxHgAYQzsSLI4Y
939 2RskkQc3xmgs3peuYtXrDU
940 2RSYXgNps0QUE12EuIUGHL
941 2rtIxn1zJEFMN2naD8VofS
942 2rUHBIfbMBB92n1gSfSqwF
943 2RVD9taXtOtcapahld30E2
944 2RxLdgkHPUbUeK0XjkTxjq
945 2rYYg1HMvlNT9A1SdjSYlx
946 2s9SV3uTSdEf2Z3pSfou6d
947 2SasoXZyv82yYgHiVOvxQn
948 2SDdwRprpaY6IXVjYbF0CX
949 2seqsVYFO7vHMPWIrAYSSf
950 2skiGld2KxenMUceC79a30
951 2SNPBo1E4duhRBXQBl6qIY
952 2spI9rsadtH5OlgMWuWX8e
953 2SRNlVul3lDW5e6EmLjWwe
9

1213 3hWRTADmmOGK9vQyqABl0R
1214 3I0FBDc1c1BLNtXWKVjmFg
1215 3ICZte49haM8ID7H6LSSjb
1216 3iD5nTgCP3oRoiZi0yT7WT
1217 3iHvXVAXLpoGMWnDhpFspL
1218 3Is1ugfPG3hdFWTq15nNBV
1219 3isxH73IAtslp0gEftrwoD
1220 3iTZLNrOzz43Oi2AZakUxV
1221 3iVcZ5G6tvkXZkZKlMpIUs
1222 3iXNy7VaY9OqJFBwzhmbOh
1223 3J4IRS0Fuz7PCvCe6fH9jd
1224 3J5fFQ0PxfKQU5putEvUoM
1225 3j7jdIypVLYAWjIk1LLXR8
1226 3jA5KKyAh3RS07CHzyi0A2
1227 3JATo1wuknvNy3SCdkza8w
1228 3JaWTOUAO2r8A0l3VZqW1E
1229 3JGBlKEOLFyNsH3Hb3pyeu
1230 3JhfyWKIYR3opA4DpHxP1w
1231 3JmpgU0GiVdAb9yITZPLct
1232 3jq222kFHktVQc6dbsDPRo
1233 3JR6wXMbEwhZiTUWy3VzZ2
1234 3JsWU9n9kRR3c6piMh2E3S
1235 3JWiDGQX2eTlFvKj3Yssj3
1236 3jWnfv5QqiNCMaGUcLqGjo
1237 3KKl1mLAJyZlt5TMoH17dL
1238 3KOsU0HxbEJ69R4KYpY86S
1239 3kpH1izy0QrjIDBFbz83YD
1240 3kqC9r9ilM63AogQ47RVpu
1241 3KSLbVjE9Hdx6abOGewoj4
1242 3Ku9kSWtDQPC8bSWpHR7z0
1243 3kZUnG3jVpgwh3TQFdTv18
1244 3L2Nyi3T7XabH8EEZFLDdX
1245 3L2YOO6dDiB91M1AbQ0CCR
1246 3lCT9uo4kbt3SANCie6axE
1247 3Lf0rT3uwJKWi1TUyKZ10J
1248 3lhtmXT6FDwGOCA

1506 4BcS4LomxUnQpzx4hCaJTs
1507 4BHSjbYylfOH5WAGusDyni
1508 4BhwlQ9mTwhFGXpQvP9JTV
1509 4BjfjRRYojXuwA6qt74fAT
1510 4BlOigB3NT7DiwHgikD9C8
1511 4BSZCOHySGSwBJlzM4qaAD
1512 4By5JAbc6KNpkyTn8cn6oa
1513 4C2XA7HSJyii3qBNaVvWXh
1514 4C6um1v4oCpbfBbZNTDry8
1515 4C9AuaGVl4wEPV1VttUpHm
1516 4cAgkb0ifwn0FSHGXnr4F6
1517 4cAlWy0jWMtn5AdtZy4dHi
1518 4cE2duTd0g33sAZaFsUyji
1519 4Cfx5TGmw5Kr96AHuWFYyj
1520 4Cg8Nl5JjI4Df0AMbm18u7
1521 4cKJsZuzbNBpdxeMC9pBQG
1522 4CoSCPlKNrWli7E5kFtbcl
1523 4CrBjpasaIf70RbJ0HbCQD
1524 4cTGilpefugCNVTirkVfSm
1525 4cuHygiGZuEeaAMTh6q8H6
1526 4Cv8KLIBrN25GRIJ3aeWq2
1527 4cYixasStbsXGnhyKxHEFN
1528 4CZF7KwvFmUUDkl4P9lfYn
1529 4D6TpthzUVNTjWQqfBW5Kb
1530 4ddA4oHvhrGx196qF5ZyQe
1531 4DpVLXMJpcCUGVd8hSM15Z
1532 4DsFdZmkVhYjMAfEFctvqg
1533 4DT5ewYCLjhAVEqPMlSGKU
1534 4dX6IOb6uoZo8WKRoDCuZm
1535 4E0hCWLiAHoxTeBXiJqeFj
1536 4e5IPJxSGVWPsaxQzdRseN
1537 4EaHhUu7bZATp4W7IAwOqD
1538 4EcM86vNf0QIOGkpf4O0UP
1539 4EIzsbsVOzfLwyIvfsRQ8H
1540 4eMRFikc0gpBmP0CAjWceV
1541 4EndPA8b6FDWYp1

1799 4XwQKfh45dStVTuGH9f04W
1800 4xyPqgnOnH076SJEW2QUwF
1801 4XYQtNf5BxGeAiwR0YU2Gn
1802 4Y3RwJrwcE2BNaCEnCdJFk
1803 4y84ILALZSa4LyP6H7NVjR
1804 4yEBlYhwi1CKSsERj4epTL
1805 4yj7faZUHawdifXFZzbO80
1806 4yNnQgQlCEZDSn5eQoUMPZ
1807 4yQFG0pjY3UagWYkfIfWYg
1808 4YUSC63LMMUbaeLjcfT4Go
1809 4YyM7qJL5FdZDQP7G5Dnf5
1810 4yz5cxtyvHcD8kjxh9p8UZ
1811 4z02bBuibhxi3cU5dxUdB4
1812 4Z2rzPq2mZk7dRdWWE0oo4
1813 4z3nbTFJ52I3qhxTSLhNKE
1814 4Z7uQ0Yk8x2itRiND63i8T
1815 4Z80C8NlDd1AO4axf1O3n0
1816 4ZC6OO6P6mE3AtgZZhHECZ
1817 4zLZquLTY4COvoF1M6q3M9
1818 4ZNcd1iiFpRe1OoTVXPqgy
1819 4Zp35RIATJCwHAKEhtiSb0
1820 4zqB6PfU6tBuUkHBZWMVK0
1821 4ZZuWpuuOCxUmE08IcuiRA
1822 50JK0hBaHMWYo20S7AvKbB
1823 50O81EQ5oLaDZpW2uDf2rO
1824 50yh3Cz0xSqhKPj6O5Ytzf
1825 51G0OWdttZATq5WutoSFe1
1826 51IHib6zwehhl4oDbhMc8a
1827 51LhklOVtu2KixJMzinIV0
1828 51mK2xGop7WMDVeuHLvhWP
1829 51PDILgbGYlDPlW3MPNdGV
1830 51PO0Md0UhUWdzN4hcEuiN
1831 51t8PlDG1zO22NcT3PfCN4
1832 51u554Q943wUHm43Ob4DwV
1833 51VCXnEk8YIBuOazvbhext
1834 523HruoIx6BPW8J

2092 5T6oEvnj0EZgkEq9bV4clI
2093 5tdERSH4Mgu2dylN1rp3xr
2094 5TGYo4MrNKqIvJqgx134py
2095 5tHe4P0tNgFLG4xoEWucUq
2096 5tIHM4AZGOGqJoiZdKqNRx
2097 5Tl0HJvynZtKdSUMKbFVVX
2098 5tLEgFI1Ud3bl5wPktw7mt
2099 5TQQvGoLY2IopeOv3HeEML
2100 5TsL4BVUIMvvWXuHwl152V
2101 5tSTWhMTSGAvbJsyUP79kf
2102 5TYQonY07iJP2ULhR4ptce
2103 5U2kqLgCC9W0e30rTwghLt
2104 5ugIMEa2lVMietOcgd29tD
2105 5uKDKj6SNsK7hTzCQyHwYC
2106 5v779QwHjBQl14y3EjFulU
2107 5V8gxeMq02JqxdLs1HgacP
2108 5VBWZll59ggYjEl3taGHNm
2109 5vCK2zLPho1HZ9jTBNzMQ3
2110 5VDpQALbVNz1IkpbedQoVO
2111 5vG4yrocD3WZBMsUvtKJJL
2112 5VJhCjNMfzxbsu4Z8x0Tmc
2113 5vNi5tqMDuruhChAcYGhax
2114 5vschfpax25JTbGq4I3hCE
2115 5vVFqQihO5rGCN7VshfgIR
2116 5vVR8Z021lnY7hrHCNSP8c
2117 5Vyw16U5k4bpGaRqfzyyMo
2118 5vyXqG0AQoTPG9ZslzZUf4
2119 5VzeI5JM2y9t21JwrWAnkH
2120 5vZtkssngtvya9UpPoNaz6
2121 5W51OFnd8YkoPvl0yINEzP
2122 5W5n3usWjrq1L7m0CMUszP
2123 5w9aPPZv98kgoY3uAdYgic
2124 5WGko0XPqpVg9KVoH62wVW
2125 5wI6XbCuRDjWDKcJKQsnWU
2126 5wjf8t841zjxZb3Jobcw5p
2127 5wLYd2DPo20zEzB

2385 6M5ckAVWyGgkHBzymUX0r1
2386 6m81AWwIChF9vKcw4co7WN
2387 6mAGUsHA1TRCw2M7mkbZxz
2388 6MgGapP3EPFm9kYUvYBTZR
2389 6MjhFQORsv3ba7Yi9qKZqk
2390 6mNlcv6vQzGfRI4kEkiSM5
2391 6mSkYcnyvntZVVb3uOVt9W
2392 6Mtt0rvg195HXHEJU3LeMr
2393 6MWLG6GNRdV9qCI7LrK9BR
2394 6mYbR9Vh8KBcNEVJWBoG3N
2395 6MzIsyys9Ws1aXDjJjChNw
2396 6n0wyMxg0I5rHTjUkTaA0G
2397 6N43Qgg9jx21vfMkTYvSi3
2398 6N9iFiRDxPfc9idumtG8rT
2399 6N9JEq02Lgc5LMbNshEdfT
2400 6NeRpRTcFftvt3j1vF4Pcw
2401 6Ni8kUwX9cLgBCxBlXLZXH
2402 6njn1Fgy02IKmPGSqP0LYX
2403 6nMBg0NXX9y4639EsS8sLO
2404 6nPdtCy29uNvDnFynt1AXL
2405 6nsKGFoC9NGq5PCb6JuKuZ
2406 6nTCT2XUEJJEJ99FEXTOSO
2407 6NtmoPmjG21kb1JfoiMMXJ
2408 6o8JXE4kaydZ1AqmXLE5ir
2409 6o9Rn8ZqSgsaXSELiwOirB
2410 6oAfRCJrtwoaAS8UySiTo7
2411 6OBEKyFZgWR0eKLFMJjJG1
2412 6OC7KGUZWnfA76r9IbYsAy
2413 6OEpSYq8wdqnl01hsoQGQY
2414 6ogxNURdCpRGmtOYbECyqg
2415 6oHpLl0XdlbYJuGLDiREfe
2416 6OhWi7WLeb1DoCyYOwNLp6
2417 6oilcOya1YbTc9ibBp2bJs
2418 6OkOFxU6j0UufR97VgSnS7
2419 6Okqb0Krvl8XqFiLJsDOhG
2420 6oQJj3CBb2mx3as

2678 7GRTrWsP3yo2Fx8umJfA1X
2679 7gsbOwZoDNVgZbkm8sRvxV
2680 7gv3Z2VSvWjEA4HEqVtXtF
2681 7GYTOJWIobqOSP7sJw4E8m
2682 7h83OV6kDVBAxx4ctsMO3i
2683 7HdfEZlWq4GqS6pu4jYdxb
2684 7hDQerpfE5FgYtVKEVemwl
2685 7hgfj2oMmIoCyX9Gbb3Tti
2686 7HjdlG4YhyHfZcGCx3HY7s
2687 7HKez549fwJQDzx3zLjHKC
2688 7hnehY2vACxvGRAarPmK1o
2689 7hQ3gIPprZp08ygVc279Qj
2690 7HrzErXq3TsKOY1gmdIShB
2691 7hTZ6o1pppnZNWyAE0bjiO
2692 7hUlZfirjFpQwzoE5GFwyL
2693 7hXy7Oc0XfODlcR8ESR9up
2694 7hY5jzlcuzefRTNCOQjlo9
2695 7hyKTfURWbe2SkLMcixicI
2696 7i0VrdLCDhvgWXfcT6EQJG
2697 7iCqdpfPTVk1fw12eqfCAP
2698 7ijXITJ7C9HgpU65GHs2Zr
2699 7Im4IUlD1OBIvrzQLm1XXt
2700 7inZdInsSXimyyDEtY0YzL
2701 7iOhWWYjhhQiXzF4o4HhXN
2702 7iPabscOgrBuDkjZAvOVRD
2703 7isAYELhhKBIdffQS1wMbB
2704 7IWu3uIUIYIeafzwv582Ho
2705 7j5teB6BOk5n7K0gOxBT5G
2706 7jArWcupwt9u6woJ9EdQlM
2707 7jAxGkXU58FwNMjqoMahn7
2708 7jCOvsWfneHRULA5JKqwTV
2709 7Jnhi80ih9NBQtWsX6M6Je
2710 7jr5SBXOsySDix0eNhUa1s
2711 7JS6aAiymUcmAqEYEnCrlX
2712 7jTBodGQzYdcXbiTIz5GEe
2713 7jvLEJh9VQAMyxv

In [211]:
flat_sections = [item for section in all_sections for item in section]

In [212]:
len(flat_sections)

31185

In [216]:
insert_tuples = [tuple(x.values()) for x in flat_sections]


# Build insert statement and insert artist information into table

cols_insert = ''
str_insert = ''
for col in flat_sections[0].keys():
    cols_insert += col + ', '
    str_insert += '%s, '
cols_insert = cols_insert.strip(', ')
str_insert = str_insert.strip(', ')

insert_statemet = """INSERT INTO music.sections ({}) VALUES ({})""".format(cols_insert, str_insert)

cursor.executemany(insert_statemet, insert_tuples)
cnx.commit()

In [22]:
### should engineer a feature for number of sessions
## engineer feature for variance in loadness of sectionsd
# https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-analysis/

for key in sp.audio_analysis(gotta_have_it_uri)['sections'][0].keys():
    print (key + " float (10)")

start float (10)
duration float (10)
confidence float (10)
loudness float (10)
tempo float (10)
tempo_confidence float (10)
key float (10)
key_confidence float (10)
mode float (10)
mode_confidence float (10)
time_signature float (10)
time_signature_confidence float (10)


In [15]:
jayz_uri = 'spotify:artist:3nFkdlSjzX9mRTtwJOzDYB'
wtt_uri = sp.artist_albums(artist_id=jayz_uri)['items'][11]['uri']

In [16]:
gotta_have_it_uri = sp.album_tracks(wtt_uri)['items'][4]['uri']

In [21]:
# https://developer.spotify.com/documentation/web-api/reference/tracks/get-audio-features/
sp.audio_features(gotta_have_it_uri)[0]


{'acousticness': 0.411,
 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/6JFLZH9vCzgqnUToBdiHQb',
 'danceability': 0.74,
 'duration_ms': 140747,
 'energy': 0.574,
 'id': '6JFLZH9vCzgqnUToBdiHQb',
 'instrumentalness': 0,
 'key': 4,
 'liveness': 0.262,
 'loudness': -6.938,
 'mode': 1,
 'speechiness': 0.245,
 'tempo': 149.762,
 'time_signature': 4,
 'track_href': 'https://api.spotify.com/v1/tracks/6JFLZH9vCzgqnUToBdiHQb',
 'type': 'audio_features',
 'uri': 'spotify:track:6JFLZH9vCzgqnUToBdiHQb',
 'valence': 0.33}

In [53]:
sp.artist(jayz_uri)

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3nFkdlSjzX9mRTtwJOzDYB'},
 'followers': {'href': None, 'total': 4998282},
 'genres': ['east coast hip hop',
  'gangster rap',
  'hip hop',
  'pop rap',
  'rap',
  'southern hip hop'],
 'href': 'https://api.spotify.com/v1/artists/3nFkdlSjzX9mRTtwJOzDYB',
 'id': '3nFkdlSjzX9mRTtwJOzDYB',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/4912d27d6b01dd790313d8ef76586be6b100550f',
   'width': 640},
  {'height': 320,
   'url': 'https://i.scdn.co/image/027c5605d895158497ff377d92d69fe89dba1730',
   'width': 320},
  {'height': 160,
   'url': 'https://i.scdn.co/image/1420226e471d0e56ae4de7cb28ba2eb4de0a29ce',
   'width': 160}],
 'name': 'JAY-Z',
 'popularity': 86,
 'type': 'artist',
 'uri': 'spotify:artist:3nFkdlSjzX9mRTtwJOzDYB'}

In [54]:
sp.artist_albums(artist_id=jayz_uri)['items'][11]

{'album_group': 'album',
 'album_type': 'album',
 'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3nFkdlSjzX9mRTtwJOzDYB'},
   'href': 'https://api.spotify.com/v1/artists/3nFkdlSjzX9mRTtwJOzDYB',
   'id': '3nFkdlSjzX9mRTtwJOzDYB',
   'name': 'JAY-Z',
   'type': 'artist',
   'uri': 'spotify:artist:3nFkdlSjzX9mRTtwJOzDYB'},
  {'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
   'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
   'id': '5K4W6rqBFWDnAN6FQUkS6x',
   'name': 'Kanye West',
   'type': 'artist',
   'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}],
 'available_markets': ['CA', 'JP', 'MX', 'US'],
 'external_urls': {'spotify': 'https://open.spotify.com/album/7mCeLbChyegbRwwKK5shJs'},
 'href': 'https://api.spotify.com/v1/albums/7mCeLbChyegbRwwKK5shJs',
 'id': '7mCeLbChyegbRwwKK5shJs',
 'images': [{'height': 640,
   'url': 'https://i.scdn.co/image/c0df37da32cbe3212fbc95fd6863e2e9047b086f',
   'widt

In [55]:
sp.track(gotta_have_it_uri)

{'album': {'album_type': 'album',
  'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3nFkdlSjzX9mRTtwJOzDYB'},
    'href': 'https://api.spotify.com/v1/artists/3nFkdlSjzX9mRTtwJOzDYB',
    'id': '3nFkdlSjzX9mRTtwJOzDYB',
    'name': 'JAY-Z',
    'type': 'artist',
    'uri': 'spotify:artist:3nFkdlSjzX9mRTtwJOzDYB'},
   {'external_urls': {'spotify': 'https://open.spotify.com/artist/5K4W6rqBFWDnAN6FQUkS6x'},
    'href': 'https://api.spotify.com/v1/artists/5K4W6rqBFWDnAN6FQUkS6x',
    'id': '5K4W6rqBFWDnAN6FQUkS6x',
    'name': 'Kanye West',
    'type': 'artist',
    'uri': 'spotify:artist:5K4W6rqBFWDnAN6FQUkS6x'}],
  'available_markets': ['CA', 'JP', 'MX', 'US'],
  'external_urls': {'spotify': 'https://open.spotify.com/album/7mCeLbChyegbRwwKK5shJs'},
  'href': 'https://api.spotify.com/v1/albums/7mCeLbChyegbRwwKK5shJs',
  'id': '7mCeLbChyegbRwwKK5shJs',
  'images': [{'height': 640,
    'url': 'https://i.scdn.co/image/c0df37da32cbe3212fbc95fd6863e2e9047b086f',
    '

In [ ]:
sp.tracks()